In [265]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt, numpy as np
from mpl_toolkits.mplot3d import Axes3D
import time


In [266]:
# Simulation Parameters
length = 10 # Meters
resolution = 1000

# Firefly Parameters
vision_length = 3 #meters
vision_cone = 40 #degrees

In [267]:
class swarm:
    def __init__(self, n, spread, attraction, repulsion, randomness, sim_frames = 100):
        self.fireflies = []
        self.N = n
        self.c1 = attraction  # Attraction Scaling factor
        self.c2 = repulsion   # Repulsion scaling factor
        self.c3 = randomness  # Randomness scaling factor

        self.rep_radius = 1   # Radius of repulsion effect

        self.frames = sim_frames  # No. of frames
        self.delta = 1            # Scaling of frames

        self.pspread = spread  # Spread of initial positions (gaussian)
        self.vspread = 2  # Spread of initial velocitys (gaussian)

        # initiaized below
        self.p = None
        self.v = None
        self.o = None

    def initialize(self):
        # positions
        self.p = np.random.normal(loc=10, scale=self.pspread, size=(3, self.N))
        # velocities
        self.v = self.vspread*np.random.randn(3, self.N)
        self.v = self.v / np.linalg.norm(self.v, axis=0)
        #self.fig = plt.figure()
        #self.ax = self.fig.add_subplot(111)

    def __repr__(self):
        return self.p
    
    def graph(self):
        x = self.p[0,:]
        y = self.p[1,:]
        z = self.p[2,:]

        fig = plt.figure(figsize = (10, 7))
        ax = plt.axes(projection ="3d")
    
        # Creating plot
        ax.scatter3D(x,y,z, color = "green")
        plt.title("Fireflies in Space")

        ax.set_xlim3d(0,20)
        ax.set_ylim3d(0,20)
        ax.set_zlim3d(0,20)
    


    def ff_in_vision(self, n):
        ## KURT ##
        ## For now, develop this method that will return the indexes of fireflies
        ## in the visual range of firefly index n. i.e. self.p[:,n]
        ## you'll need to determine the orientation of the firefliy with the
        ## 3D velocity vectors in self.v[:,n], and then extend out a 3D cone
        ## with some arbitrary parameters (I'm thinking h = 1m, r = .3m?)
        ## reuturn all indicies for fireflies that fall within that cone
        ## try to make it computationally efficient.
        return None

    def center_mass(self):  # Calculate center of gravity of all fireflies
        pos_sums = 0
        for i in range(self.N):
            pos_sums += self.p[:,i]
        com = pos_sums / self.N
        return( com.reshape((3,1)) )
    
    def simulate(self):    # Simulate firefly swarm
        for _ in range(self.frames): # Iterate through all frames
            # Attraction to center of gravity
            center = self.center_mass()
            attraction = center - self.p

            # Randomness
            randomness = np.random.normal(loc=0, scale=1, size=(3, self.N))

            # Pre-allocate repulsion vector
            repulsion = np.zeros((3, self.N))

            for i in range(self.N):
                ## Find all fireflies within range of ith 
                temp = np.delete(self.v, i, axis=1)

                # Selct all columns where euclidean distance is less than the rep_radius
                distance = np.linalg.norm(temp - self.v[:,i].reshape(3,1), axis = 0)

                # If there are no nearby fireflies, skip
                if distance.shape[0] == 0:
                    continue  
                # select vector columns & distances that meet distance criteria
                slices = temp[:, np.where(distance < self.rep_radius)[0]]
                distance_slice = distance[np.where(distance < self.rep_radius)]
                # Scale slices by inverse of distance (farther away = less strong effect)
                slices *= (1/distance_slice)  # Scale slices by distance
                # Find the average repulsion 
                repulsion[:,i] = np.mean(self.v[:,i].reshape(3,1) - slices, axis = 1).reshape(3,)
            # Normalize all vectors
            attraction = attraction / np.linalg.norm(attraction, axis=0)
            repulsion = repulsion / np.linalg.norm(repulsion, axis=0)
            randomness = randomness / np.linalg.norm(randomness, axis=0)
            # Update velocities
            self.v = self.v + self.c1*attraction + self.c2*repulsion + self.c3*randomness
            # Update positions
            self.p = self.p + self.v * self.delta
            # Graph
            self.graph()

In [268]:
n_agents = 50
c1 = 0.1  # attraction
c2 = 0.001  # repulsion
c3 = 0.01  # randomness
c4 = 3.5

fireflies = swarm(n = n_agents, spread = c4, attraction = c1, repulsion = c2, randomness = c3)
fireflies.initialize()
#fireflies.simulate()
#fireflies.graph()